# Q1 — Net revenue, cohorts & rolling features

This notebook is **self-contained** (datasets built from dictionaries; no external files).  
Complete each task using **vectorized Pandas** operations only — **no Python loops**.

**Datasets:** orders & returns (see below)  
**Reference date:** 2025-03-31

---

## Tasks
1. **City Cleaning** → Normalize messy city strings into canonical names: `New York`, `Boston`, `Philadelphia`, `Newark`.  
   - Create `city_clean` using string ops only (replace `_`/`-`, collapse spaces, title-case, etc.).  
   - Treat all variants of New York (e.g., `new_york`, `new-york`, `New  York`, mixed case) as **New York**.

2. **Net Revenue per Order**  
   - `gross = qty * unit_price`  
   - Exclude `status == 'cancelled'` from revenue computations.  
   - Join `returns`: orders present in `returns` must have **`net_revenue = 0`**.  
   - Create `net_revenue` column.


**Constraints:** Use `merge`, `groupby`, `pivot_table`/`crosstab`, vectorized datetime math, rolling/date filtering. **No loops.**


In [3]:
# Imports
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')


## Build datasets from dictionaries (run this first)

In [4]:
# Orders and returns as dictionaries -> DataFrames
orders_dict = {
    "order_id": [1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015],
    "customer_id": ["C01","C02","C01","C03","C02","C04","C01","C05","C02","C03","C04","C01","C05","C02","C06"],
    "order_date": ["2025-01-03","2025-01-05","2025-01-18","2025-02-02","2025-02-08","2025-02-15","2025-02-16","2025-02-20","2025-03-01","2025-03-03","2025-03-07","2025-03-11","2025-03-14","2025-03-19","2025-03-21"],
    "product": ["Keyboard","Laptop","Mouse","Monitor","Headset","Laptop","Mouse","Dock","Monitor","Keyboard","Mouse","Laptop","Headset","Laptop","Monitor"],
    "category": ["Peripherals","Computers","Peripherals","Displays","Peripherals","Computers","Peripherals","Peripherals","Displays","Peripherals","Peripherals","Computers","Peripherals","Computers","Displays"],
    "qty": [1,1,2,1,1,1,1,1,2,1,3,1,2,1,1],
    "unit_price": [60,950,25,220,80,1050,25,120,210,65,24,990,78,980,230],
    "coupon_code": [None,"NY10",None,"NY10",None,None,None,None,None,None,None,"SPRING10",None,None,None],
    "city": ["new york","NEW YORK","New  York","boston","new_york","Boston","new york","philadelphia","New York","BOSTON","boston","New york","PHILADELPHIA","new-york","newark"],
    "status": ["completed","completed","completed","completed","completed","completed","cancelled","completed","completed","completed","completed","completed","completed","completed","completed"]
}

returns_dict = {
    "order_id": [1004,1009],
    "return_date": ["2025-02-15","2025-03-10"],
    "reason": ["Dead on arrival","Broken pixels"]
}

orders = pd.DataFrame(orders_dict)
returns = pd.DataFrame(returns_dict)
orders["order_date"] = pd.to_datetime(orders["order_date"])
returns["return_date"] = pd.to_datetime(returns["return_date"])

print("orders shape:", orders.shape)
print("returns shape:", returns.shape)
orders.head()

orders shape: (15, 10)
returns shape: (2, 3)


,order_id,customer_id,order_date,product,category,qty,unit_price,coupon_code,city,status
0,1001,C01,2025-01-03,Keyboard,Peripherals,1,60,None,new york,completed
1,1002,C02,2025-01-05,Laptop,Computers,1,950,NY10,NEW YORK,completed
2,1003,C01,2025-01-18,Mouse,Peripherals,2,25,None,New York,completed
3,1004,C03,2025-02-02,Monitor,Displays,1,220,NY10,boston,completed
4,1005,C02,2025-02-08,Headset,Peripherals,1,80,None,new_york,completed


## Task 1 — City Cleaning → `city_clean`

In [5]:
# TODO: create 'city_clean' in orders
# Requirements:
# - Use only string operations (lower(), replace underscores/hyphens, collapse multiple spaces, strip, title-case)
# - Treat any variant of New York as 'New York'
# - Canonical set: 'New York','Boston','Philadelphia','Newark'

# Your code here
orders["city_clean"] = (
    orders["city"]
    .str.lower()
    .str.replace("_", " ")
    .str.replace("-", " ")
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
    .str.title()
)
orders.loc[orders["city_clean"].str.contains("New York", case=False), "city_clean"] = "New York"

# Display preview
orders[["city_clean"]].head()



,city_clean
0,New York
1,New York
2,New York
3,Boston
4,New York


## Task 2 — Net Revenue per order

In [9]:
# TODO:
# - Create 'gross = qty * unit_price'
# - Exclude 'cancelled' from revenue computations (work on a filtered copy)
# - Left-join returns; if returned -> net_revenue = 0 else gross
# - Keep columns: order_id, customer_id, order_date, city_clean, qty, unit_price, gross, returned, net_revenue

# Your code here
valid = orders[orders["status"] != "cancelled"].copy()
valid["gross"] = valid["qty"] * valid["unit_price"]

valid = valid.merge(
    returns[["order_id"]].assign(returned=True),
    on="order_id",
    how="left"
)
valid["returned"] = valid["returned"].fillna(False)
valid["net_revenue"] = np.where(valid["returned"], 0, valid["gross"])


valid = valid[
    [
        "order_id", "customer_id", "order_date", "city_clean",
        "qty", "unit_price", "gross", "returned", "net_revenue"
    ]
]

# Preview
valid.sort_values("order_id").head(10)

/var/folders/vx/q1wb3y9j7sj6b13bvtyv2rtr0000gn/T/ipykernel_29833/181919011.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  valid["returned"] = valid["returned"].fillna(False)


,order_id,customer_id,order_date,city_clean,qty,unit_price,gross,returned,net_revenue
0,1001,C01,2025-01-03,New York,1,60,60,False,60
1,1002,C02,2025-01-05,New York,1,950,950,False,950
2,1003,C01,2025-01-18,New York,2,25,50,False,50
3,1004,C03,2025-02-02,Boston,1,220,220,True,0
4,1005,C02,2025-02-08,New York,1,80,80,False,80
5,1006,C04,2025-02-15,Boston,1,1050,1050,False,1050
6,1008,C05,2025-02-20,Philadelphia,1,120,120,False,120
7,1009,C02,2025-03-01,New York,2,210,420,True,0
8,1010,C03,2025-03-03,Boston,1,65,65,False,65
9,1011,C04,2025-03-07,Boston,3,24,72,False,72
